In [10]:
import ufl
import gmsh
import numpy as np
from dolfinx import io,fem,mesh
from mpi4py import MPI
gmsh.initialize()

In [11]:
#read in an adcirc mesh and save some stuff
def ADCIRC_mesh_gen(file_path):
    #specify file path as a string, either absolute or relative to where script is run
    #only compatible for adcirc fort.14 format
    adcirc_mesh=open(file_path,'r')
    title=adcirc_mesh.readline()

    #NE number of elements, NP number of grid points
    NE,NP=adcirc_mesh.readline().split()
    NE=int(NE)
    NP=int(NP)

    #initiate data structures
    NODENUM=np.zeros(NP)
    LONS=np.zeros(NP)
    LATS=np.zeros(NP)
    DPS=np.zeros(NP)
    ELEMNUM=np.zeros(NE)
    NM = np.zeros((NE,3)) #stores connectivity at each element

    #read node information line by line 
    for i in range(NP):
        NODENUM[i], LONS[i], LATS[i], DPS[i] = adcirc_mesh.readline().split()
    #read in connectivity
    for i in range(NE):
        ELEMNUM[i], DUM, NM[i,0],NM[i,1], NM[i,2]=adcirc_mesh.readline().split()

    #(we need to shift nodenum down by 1)
    ELEMNUM=ELEMNUM-1
    NM=NM-1
    NODENUM=NODENUM-1

    #close file
    adcirc_mesh.close()
    return NODENUM,[LONS,LATS],ELEMNUM,NM
    

In [12]:
NODENUM,[LONS,LATS],ELEMNUM,NM = ADCIRC_mesh_gen("depth2.grd")
#can read in xdmf mesh file
#filename = 'mesh_test.xdmf'
#encoding= io.XDMFFile.Encoding.HDF5
#with io.XDMFFile(MPI.COMM_WORLD, filename, "r", encoding=encoding) as file:
#    mesh2 = file.read_mesh()

In [19]:
gdim, shape, degree = 2, "triangle", 1
cell = ufl.Cell(shape, geometric_dimension=gdim)
element = ufl.VectorElement("Lagrange", cell, degree)
domain = ufl.Mesh(element)
coords = np.array(list(zip(LONS,LATS)))


x = np.array([[0., 0., 0.], [0., 1., 0.], [1., 1., 0.]])
cells = np.array([[0, 1, 2]], dtype=np.int64)
domain1 = mesh.create_mesh(MPI.COMM_WORLD, NM, coords, domain)

V = fem.FunctionSpace(domain1, ("P", 1))
dof_coords = V.tabulate_dof_coordinates()
#assign depth as a function of y
depth = 20 - dof_coords[:,1]/200
f = fem.Function(V)
f.x.array[:] = depth


In [20]:
#plot the mesh and a function on top of it
fname = 'depth_mesh/solution'
xdmf = io.XDMFFile(domain1.comm, fname+".xdmf", "w")
xdmf.write_mesh(domain1)
xdmf.write_function(f)
xdmf.close()